# ルートを評価するために解くLPについて
- ルートが1つ与えられた後、そのルートを評価するためにLPを解く(ルートが実行可能であるとは限らない)

<!--- 各制約の違反度を変数とし、それらに重みをかけて足し合わせた関数の最小化問題とする-->
- 車両が各顧客へ到着する時刻を変数とし、その合計を最小化する問題とする
- 制約は、容量制約と時間枠制約とする
    - 容量制約は、ある区間における2つの関数の積分値(面積)の大小を比較するという形で表す(車両の積荷の量を表す区分線形関数の\[x0, xn\]までの積分値と最大容量を表す線形関数の\[x0, xn\]までの積分値)
    - 時間枠制約は、通常のVRPの定式化と同様に表す


# 前準備

## 問題例の読み込み
使用した問題例 : https://neo.lcc.uma.es/vrp/vrp-instances/description-for-files-of-solomons-instances/

In [ ]:
class Customer():
    def __init__(self, x, y, d, e, l, s):
        self.x = x
        self.y = y
        self.d = d
        self.e = e
        self.l = l
        self.s = s

In [ ]:
import glob

files = glob.glob("./solomon_25/*")
for file in files:
    print(file)

In [ ]:
# 問題例の選択
instance = files[0]

In [ ]:
Customers = {}
with open(instance, mode="r") as f:
    for index,line in enumerate(f):
        #print(line)
        l = []
        if index==4:
            line_s = [s.strip("\n") for s in line.split("\t")][0]
            for s in line_s.split(" "):
                if s!="":
                    l.append(int(s))
            K, Q = l
        elif 9<=index<=34:
            line_s = [s.strip("\n") for s in line.split("\t")][0]
            for s in line_s.split(" "):
                if s!="":
                    l.append(int(s))
            Customers[l[0]] = Customer(*l[1:])

In [ ]:
Customers["depot"] = Customer(0,0,0,0,1000,0)

In [ ]:
print("\t", list(vars(Customers["depot"]).keys()))
for i in Customers.keys():
    print(i, end="\t")
    for key, val in vars(Customers[i]).items():
        print(val, end=" ")
    print()

## ルートの作成

In [ ]:
# 入力
## 顧客
C={} # 客の座標を保存する辞書
TW={} # 客の時間枠を保存する辞書
demand={} # 客の要求量(正の値は集荷，負の値は配達)
S={} # 客のサービス時間

for i in Customers:
    x, y = Customers[i].x, Customers[i].y
    C[i] = (x,y)
    e, l = Customers[i].e, Customers[i].l
    TW[i] = (e,l)
    demand[i] = (Customers[i].d, )
    S[i] = Customers[i].s
    """name="c"+str(i)
    x, y = Customers[i].x, Customers[i].y
    C[name] = (x,y)
    e, l = Customers[i].e, Customers[i].l
    TW[name] = (e,l)
    demand[name] = (Customers[i].d, )
    S[name] = Customers[i].s"""
#C["depot"]=(0,0)

## 車両
M = K//2 # number of vehicles
capacities = [(Q*2, ) for k in range(M)] # capacity of vehicle

In [ ]:
print("Customer",C)
print("Time Window",TW)
print("Demand",demand)
print("Service Time",S)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(7,7))

G=nx.DiGraph()
nx.draw_networkx(G,pos=C,nodelist=[i for i in C if i != "depot"],node_color="y",node_size=50,with_labels=True,edge_color="k",width=1)
nx.draw_networkx(G,pos=C,nodelist=["depot"],node_color="blue",node_shape='s',alpha=0.5,node_size=100,with_labels=True,edge_color="k",width=1)

plt.show()

In [ ]:
# 距離関数の定義
#def Distance(t1,t2):
#    return ((t1[0]-t2[0])**2+(t1[1]-t2[1])**2)**(0.5)
def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(0.5)

# ソルバーの読み込み
import sys
sys.path.append('..')

#from vrplib.vrp_d_1m1_t_model import *
import vrplib.vrp_d_1m1_t_model as vrp

# ソルバーの実行
model = vrp.Model(file.split("/")[-1].split(".")[0]) # モデルインスタンスの生成

## 客インスタンスの生成
for i in C:
    if i == "depot":
        continue
    model += vrp.Customer(i,demand=demand[i],timewindow=TW[i],servicetime=S[i])

## 車両インスタンスの生成
for k in range(M):
    model += vrp.Vehicle("v"+str(k),capacity=capacities[k])
    
## 枝インスタンスの生成
for i in C:
    for j in C:
        if i!=j:
            dist = time = distance(*C[i],*C[j])
            model += vrp.Edge(i,j,dist,time)

In [ ]:
## 最適化の実行
obj=model.optimize(IterLimit=10000,TimeLimit=10,Verbose=False,OutputFlag=False)

## 得られた解の表示
print("objective value =",obj)
for v in model.vehiclesL:
    print()
    print(v)
    print(list(map(lambda x:x.name,v.routing[1:-1])))    
    for iv in map(lambda x:x.name,v.routing[1:-1]):
        print(iv)

## ルートを1つ選ぶ

In [ ]:
# Vehicle 6の巡回路を選んでいる
tour = [12, 3, 24, 25]#+["depot"]

## その他作業

In [ ]:
# 使用した問題例の名称を保存
tmp = []
for s in instance[-1:0:-1]:
    if s=='/':
        break
    tmp.append(s)
instance_name = "".join(tmp[-1:0:-1])
print(instance_name)

# 前準備

## instances.pyからインスタンスを得る

In [1]:
import instances
Customers = instances.Customers
tour = instances.tour

In [2]:
instance_name = "C" + str(len(Customers)-1)

In [3]:
tour[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
tour[-10:]

[6312, 6313, 6314, 6315, 6316, 6317, 6318, 6319, 6320, 'depot']

# 問題を解く
2つの手法に対し、計算時間を比較

## 入力する行列、ベクトルの作成

In [5]:
def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(0.5)

In [6]:
def make_preceding_constr(tour, Customers, Ax, Ap, b, c):
    # ルート内の顧客の順序に関する制約
    for index, i in enumerate(tour):
        try:
            i_next = tour[index+1]
        except:
            i_next = "depot"
            continue
        Ax.append([1 if target==i else -1 if target==i_next else 0 for target in tour])
        Ap.append([0 for _ in range(len(tour))])
        b.append(-distance(Customers[i].x, Customers[i].y, Customers[i_next].x, Customers[i_next].y))
    return Ax, Ap, b, c

In [7]:
def make_tw_constr(tour, Customers, Ax, Ap, b, c):
    # 時間枠制約
    for index, i in enumerate(tour):
        Ax.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        b.append(-Customers[i].e)
        Ax.append([1 if i==target else 0 for index_, target in enumerate(tour)])
        Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        b.append(Customers[i].l)
    return Ax, Ap, b, c

In [8]:
def make_inputs(tour, Customers):
    import numpy as np
    Ax, Ap, b, c = [], [], [], [1 for _ in range(len(tour))]
    # ルート内の顧客の順序に関する制約
    Ax, Ap, b, c = make_preceding_constr(tour, Customers, Ax, Ap, b, c)
    print("Preceding constraints are done.")
    # 時間枠制約
    Ax, Ap, b, c = make_tw_constr(tour, Customers, Ax, Ap, b, c)
    print("Time-window constarints are done.")
    # ndarrayに変換
    Ax = np.array(Ax)
    Ap = np.array(Ap)
    b = np.array(b)
    c = np.array(c)
    return Ax, Ap, b, c

## 主問題を解く関数の定義
与えられた定数を元にLPのモデルを作成した上でそれを解き、最適解を返す関数

In [9]:
def solve_primal(Ax, Ap, b, c, instance_name, num_vars):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    # 定数を設定　←　入力として与えられる
    # 変数を設定
    """
    x_i : 顧客iへ車両が到着する時刻を表す変数
    p_i : 顧客iの時間枠の違反度を表す変数
    """
    x = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="x")
    p = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="p")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(c.T @ p, sense=gp.GRB.MINIMIZE)
    
    # 制約条件を設定
    m.addConstr(Ax @ x + Ap @ p <= b, name="c")

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # 最適化
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"車両が顧客{i}に到着する時刻は、{x[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')
        
    return m

## 双対問題を解く関数の定義

In [10]:
def solve_dual(Ax, Ap, b, c, instance_name, num_vars, basis_like):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    
    # 変数を設定
    """
    y_i : 主問題における制約式iの潜在価値
    """
    y = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="y")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(-1 * b.T @ y, sense=gp.GRB.MAXIMIZE)
    
    # 制約条件を設定
    m.addConstr(Ax.T @ y >= 0, name="c1")
    m.addConstr(Ap.T @ y + c >= 0, name="c2")
    m.addConstr(y >= 0, name="c3")

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # ホットスタートの使用
    for i in range(num_vars):
        y[i].PStart = basis_like[i]
    
    # 最適化
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"主問題における制約{i}の潜在価格は、{y[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')
        
    return m

## ①全体を1つのLPとして解くveb.

### 全体のPrimalを解く

In [11]:
Ax, Ap, b, c = make_inputs(tour, Customers)

Preceding constraints are done.
Time-window constarints are done.


In [12]:
# Gurobiによって最適解を求める
whole_x = solve_primal(Ax, Ap, b, c, instance_name+"whole_x", len(tour))

Using license file /Users/okamoto/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 18965 rows, 12644 columns and 37930 nonzeros
Model fingerprint: 0xa123105c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 12583 rows and 6264 columns
Presolve time: 0.17s
Presolved: 6382 rows, 6380 columns, 12764 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 32
 AA' NZ     : 1.527e+04
 Factor NZ  : 2.243e+04 (roughly 5 MBytes of memory)
 Factor Ops : 2.991e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.50373673e+08 -1.00000000e+03  1.06e+04 0.00e+00  1.

車両が顧客144に到着する時刻は、[882.88926012]
車両が顧客145に到着する時刻は、[887.88926012]
車両が顧客146に到着する時刻は、[892.1319008]
車両が顧客147に到着する時刻は、[894.36796878]
車両が顧客148に到着する時刻は、[903.80194991]
車両が顧客149に到着する時刻は、[909.45880416]
車両が顧客150に到着する時刻は、[911.45880416]
車両が顧客151に到着する時刻は、[917.54156669]
車両が顧客152に到着する時刻は、[919.77763467]
車両が顧客153に到着する時刻は、[930.54796429]
車両が顧客154に到着する時刻は、[935.93312909]
車両が顧客155に到着する時刻は、[939.53868037]
車両が顧客156に到着する時刻は、[942.53868037]
車両が顧客157に到着する時刻は、[947.53868037]
車両が顧客158に到着する時刻は、[954.74978292]
車両が顧客159に到着する時刻は、[962.5600326]
車両が顧客160に到着する時刻は、[972.60990822]
車両が顧客161に到着する時刻は、[976.21545949]
車両が顧客162に到着する時刻は、[981.6006243]
車両が顧客163に到着する時刻は、[985.84326499]
車両が顧客164に到着する時刻は、[991.67421688]
車両が顧客165に到着する時刻は、[1002.30436269]
車両が顧客166に到着する時刻は、[1009.37543051]
車両が顧客167に到着する時刻は、[1017.43768825]
車両が顧客168に到着する時刻は、[1022.53670777]
車両が顧客169に到着する時刻は、[1027.53670777]
車両が顧客170に到着する時刻は、[1033.6194703]
車両が顧客171に到着する時刻は、[1042.83901475]
車両が顧客172に到着する時刻は、[1045.07508273]
車両が顧客173に到着する時刻は、[1050.07508273]
車両が顧客174に到着する時刻は、[1054.07508273]
車両

車両が顧客481に到着する時刻は、[2867.03800702]
車両が顧客482に到着する時刻は、[2869.27407499]
車両が顧客483に到着する時刻は、[2874.27407499]
車両が顧客484に到着する時刻は、[2879.27407499]
車両が顧客485に到着する時刻は、[2881.51014297]
車両が顧客486に到着する時刻は、[2884.67242063]
車両が顧客487に到着する時刻は、[2893.61669254]
車両が顧客488に到着する時刻は、[2897.85933323]
車両が顧客489に到着する時刻は、[2898.85933323]
車両が顧客490に到着する時刻は、[2901.85933323]
車両が顧客491に到着する時刻は、[2904.09540121]
車両が顧客492に到着する時刻は、[2913.31494566]
車両が顧客493に到着する時刻は、[2914.72915923]
車両が顧客494に到着する時刻は、[2926.04286773]
車両が顧客495に到着する時刻は、[2935.47684886]
車両が顧客496に到着する時刻は、[2944.91082999]
車両が顧客497に到着する時刻は、[2946.32504355]
車両が顧客498に到着する時刻は、[2955.54458801]
車両が顧客499に到着する時刻は、[2964.14691328]
車両が顧客500に到着する時刻は、[2964.14691328]
車両が顧客501に到着する時刻は、[2969.97786517]
車両が顧客502に到着する時刻は、[2972.8062923]
車両が顧客503に到着する時刻は、[2981.29157367]
車両が顧客504に到着する時刻は、[2989.77685504]
車両が顧客505に到着する時刻は、[2999.21083618]
車両が顧客506に到着する時刻は、[3002.21083618]
車両が顧客507に到着する時刻は、[3006.3339418]
車両が顧客508に到着する時刻は、[3013.61405169]
車両が顧客509に到着する時刻は、[3019.69681422]
車両が顧客510に到着する時刻は、[3027.75907197]
車両が顧客511に到着す

車両が顧客734に到着する時刻は、[4173.18330982]
車両が顧客735に到着する時刻は、[4174.59752339]
車両が顧客736に到着する時刻は、[4181.30572732]
車両が顧客737に到着する時刻は、[4185.42883294]
車両が顧客738に到着する時刻は、[4186.42883294]
車両が顧客739に到着する時刻は、[4196.86913945]
車両が顧客740に到着する時刻は、[4199.86913945]
車両が顧客741に到着する時刻は、[4208.1153507]
車両が顧客742に到着する時刻は、[4210.94377783]
車両が顧客743に到着する時刻は、[4215.06688345]
車両が顧客744に到着する時刻は、[4224.50086459]
車両が顧客745に到着する時刻は、[4225.50086459]
車両が顧客746に到着する時刻は、[4228.32929171]
車両が顧客747に到着する時刻は、[4232.80142767]
車両が顧客748に到着する時刻は、[4242.02097212]
車両が顧客749に到着する時刻は、[4256.16310775]
車両が顧客750に到着する時刻は、[4258.99153487]
車両が顧客751に到着する時刻は、[4267.53553862]
車両が顧客752に到着する時刻は、[4273.36649051]
車両が顧客753に到着する時刻は、[4274.36649051]
車両が顧客754に到着する時刻は、[4280.36649051]
車両が顧客755に到着する時刻は、[4283.36649051]
車両が顧客756に到着する時刻は、[4290.07469444]
車両が顧客757に到着する時刻は、[4295.90564634]
車両が顧客758に到着する時刻は、[4295.90564634]
車両が顧客759に到着する時刻は、[4300.37778229]
車両が顧客760に到着する時刻は、[4308.92178604]
車両が顧客761に到着する時刻は、[4312.52733732]
車両が顧客762に到着する時刻は、[4319.59840513]
車両が顧客763に到着する時刻は、[4322.76068279]
車両が顧客764に到着

車両が顧客1007に到着する時刻は、[5709.95794669]
車両が顧客1008に到着する時刻は、[5719.17749115]
車両が顧客1009に到着する時刻は、[5726.24855896]
車両が顧客1010に到着する時刻は、[5733.52866885]
車両が顧客1011に到着する時刻は、[5737.52866885]
車両が顧客1012に到着する時刻は、[5742.91383366]
車両が顧客1013に到着する時刻は、[5745.74226078]
車両が顧客1014に到着する時刻は、[5750.21439674]
車両が顧客1015に到着する時刻は、[5757.42549929]
車両が顧客1016に到着する時刻は、[5760.58777695]
車両が顧客1017に到着する時刻は、[5765.0599129]
車両が顧客1018に到着する時刻は、[5769.0599129]
車両が顧客1019に到着する時刻は、[5774.0599129]
車両が顧客1020に到着する時刻は、[5780.14267543]
車両が顧客1021に到着する時刻は、[5783.74822671]
車両が顧客1022に到着する時刻は、[5794.04385685]
車両が顧客1023に到着する時刻は、[5802.2900681]
車両が顧客1024に到着する時刻は、[5808.12102]
車両が顧客1025に到着する時刻は、[5813.12102]
車両が顧客1026に到着する時刻は、[5822.9698778]
車両が顧客1027に到着する時刻は、[5825.9698778]
車両が顧客1028に到着する時刻は、[5834.03213555]
車両が顧客1029に到着する時刻は、[5835.44634911]
車両が顧客1030に到着する時刻は、[5841.10320336]
車両が顧客1031に到着する時刻は、[5852.76510715]
車両が顧客1032に到着する時刻は、[5855.59353427]
車両が顧客1033に到着する時刻は、[5856.59353427]
車両が顧客1034に到着する時刻は、[5858.82960225]
車両が顧客1035に到着する時刻は、[5869.26990876]
車両が顧客1036に到着する時刻は、[5874.36

...

車両が顧客5166に到着する時刻は、[29394.10221725]
車両が顧客5167に到着する時刻は、[29397.70776852]
車両が顧客5168に到着する時刻は、[29405.5180182]
車両が顧客5169に到着する時刻は、[29410.61703771]
車両が顧客5170に到着する時刻は、[29416.44798961]
車両が顧客5171に到着する時刻は、[29421.44798961]
車両が顧客5172に到着する時刻は、[29430.93482259]
車両が顧客5173に到着する時刻は、[29436.3199874]
車両が顧客5174に到着する時刻は、[29443.02819133]
車両が顧客5175に到着する時刻は、[29451.97246324]
車両が顧客5176に到着する時刻は、[29454.20853122]
車両が顧客5177に到着する時刻は、[29457.81408249]
車両が顧客5178に到着する時刻は、[29460.97636015]
車両が顧客5179に到着する時刻は、[29467.05912268]
車両が顧客5180に到着する時刻は、[29474.67489579]
車両が顧客5181に到着する時刻は、[29484.57439073]
車両が顧客5182に到着する時刻は、[29488.69749635]
車両が顧客5183に到着する時刻は、[29495.02205167]
車両が顧客5184に到着する時刻は、[29501.1048142]
車両が顧客5185に到着する時刻は、[29511.1048142]
車両が顧客5186に到着する時刻は、[29520.32435866]
車両が顧客5187に到着する時刻は、[29530.2238536]
車両が顧客5188に到着する時刻は、[29532.2238536]
車両が顧客5189に到着する時刻は、[29537.2238536]
車両が顧客5190に到着する時刻は、[29544.43495615]
車両が顧客5191に到着する時刻は、[29553.92178913]
車両が顧客5192に到着する時刻は、[29560.99285694]
車両が顧客5193に到着する時刻は、[29563.22892492]
車両が顧客5194に到着する時刻は、[29565.22

車両が顧客5406に到着する時刻は、[30822.1779338]
車両が顧客5407に到着する時刻は、[30827.1779338]
車両が顧客5408に到着する時刻は、[30830.78348507]
車両が顧客5409に到着する時刻は、[30835.02612576]
車両が顧客5410に到着する時刻は、[30841.02612576]
車両が顧客5411に到着する時刻は、[30847.42925]
車両が顧客5412に到着する時刻は、[30858.7429585]
車両が顧客5413に到着する時刻は、[30865.81402631]
車両が顧客5414に到着する時刻は、[30876.44417212]
車両が顧客5415に到着する時刻は、[30877.44417212]
車両が顧客5416に到着する時刻は、[30884.51523993]
車両が顧客5417に到着する時刻は、[30888.12079121]
車両が顧客5418に到着する時刻は、[30895.12079121]
車両が顧客5419に到着する時刻は、[30901.52391545]
車両が顧客5420に到着する時刻は、[30909.7701267]
車両が顧客5421に到着する時刻は、[30912.00619467]
車両が顧客5422に到着する時刻は、[30915.16847233]
車両が顧客5423に到着する時刻は、[30919.64060829]
車両が顧客5424に到着する時刻は、[30930.2707541]
車両が顧客5425に到着する時刻は、[30933.2707541]
車両が顧客5426に到着する時刻は、[30938.65591891]
車両が顧客5427に到着する時刻は、[30938.65591891]
車両が顧客5428に到着する時刻は、[30941.48434603]
車両が顧客5429に到着する時刻は、[30950.02834978]
車両が顧客5430に到着する時刻は、[30952.26441776]
車両が顧客5431に到着する時刻は、[30959.47552031]
車両が顧客5432に到着する時刻は、[30970.24584992]
車両が顧客5433に到着する時刻は、[30971.24584992]
車両が顧客5434に到着する時刻は、[30981.2458

車両が顧客5647に到着する時刻は、[32211.97915619]
車両が顧客5648に到着する時刻は、[32216.97915619]
車両が顧客5649に到着する時刻は、[32222.81010808]
車両が顧客5650に到着する時刻は、[32229.81010808]
車両が顧客5651に到着する時刻は、[32237.87236583]
車両が顧客5652に到着する時刻は、[32244.27549007]
車両が顧客5653に到着する時刻は、[32246.51155805]
車両が顧客5654に到着する時刻は、[32256.95186456]
車両が顧客5655に到着する時刻は、[32260.11414222]
車両が顧客5656に到着する時刻は、[32263.27641988]
車両が顧客5657に到着する時刻は、[32273.27641988]
車両が顧客5658に到着する時刻は、[32277.27641988]
車両が顧客5659に到着する時刻は、[32279.27641988]
車両が顧客5660に到着する時刻は、[32287.82042362]
車両が顧客5661に到着する時刻は、[32291.82042362]
車両が顧客5662に到着する時刻は、[32292.82042362]
車両が顧客5663に到着する時刻は、[32300.03152617]
車両が顧客5664に到着する時刻は、[32307.31163606]
車両が顧客5665に到着する時刻は、[32314.59174595]
車両が顧客5666に到着する時刻は、[32318.71485158]
車両が顧客5667に到着する時刻は、[32329.48518119]
車両が顧客5668に到着する時刻は、[32330.89939475]
車両が顧客5669に到着する時刻は、[32341.09743378]
車両が顧客5670に到着する時刻は、[32347.50055802]
車両が顧客5671に到着する時刻は、[32349.736626]
車両が顧客5672に到着する時刻は、[32357.79888374]
車両が顧客5673に到着する時刻は、[32366.28416512]
車両が顧客5674に到着する時刻は、[32374.82816886]
車両が顧客5675に到着する時刻は、[323

車両が顧客5920に到着する時刻は、[33732.24694981]
車両が顧客5921に到着する時刻は、[33733.66116337]
車両が顧客5922に到着する時刻は、[33743.14799635]
車両が顧客5923に到着する時刻は、[33746.31027401]
車両が顧客5924に到着する時刻は、[33755.797107]
車両が顧客5925に到着する時刻は、[33759.92021262]
車両が顧客5926に到着する時刻は、[33762.1562806]
車両が顧客5927に到着する時刻は、[33768.55940484]
車両が顧客5928に到着する時刻は、[33770.55940484]
車両が顧客5929に到着する時刻は、[33770.55940484]
車両が顧客5930に到着する時刻は、[33780.04623782]
車両が顧客5931に到着する時刻は、[33788.10849556]
車両が顧客5932に到着する時刻は、[33796.65249931]
車両が顧客5933に到着する時刻は、[33805.19650306]
車両が顧客5934に到着する時刻は、[33807.43257103]
車両が顧客5935に到着する時刻は、[33813.26352293]
車両が顧客5936に到着する時刻は、[33818.92037718]
車両が顧客5937に到着する時刻は、[33825.99144499]
車両が顧客5938に到着する時刻は、[33833.0625128]
車両が顧客5939に到着する時刻は、[33834.47672636]
車両が顧客5940に到着する時刻は、[33843.69627082]
車両が顧客5941に到着する時刻は、[33850.97638071]
車両が顧客5942に到着する時刻は、[33854.58193199]
車両が顧客5943に到着する時刻は、[33859.58193199]
車両が顧客5944に到着する時刻は、[33862.74420965]
車両が顧客5945に到着する時刻は、[33867.74420965]
車両が顧客5946に到着する時刻は、[33873.82697218]
車両が顧客5947に到着する時刻は、[33877.43252345]
車両が顧客5948に到着する時刻は、[33884

車両が顧客6170に到着する時刻は、[35113.72601478]
車両が顧客6171に到着する時刻は、[35115.96208276]
車両が顧客6172に到着する時刻は、[35120.96208276]
車両が顧客6173に到着する時刻は、[35126.61893701]
車両が顧客6174に到着する時刻は、[35130.22448828]
車両が顧客6175に到着する時刻は、[35139.44403274]
車両が顧客6176に到着する時刻は、[35143.04958402]
車両が顧客6177に到着する時刻は、[35151.99385593]
車両が顧客6178に到着する時刻は、[35160.93812784]
車両が顧客6179に到着する時刻は、[35168.21823773]
車両が顧客6180に到着する時刻は、[35177.27362286]
車両が顧客6181に到着する時刻は、[35180.10204999]
車両が顧客6182に到着する時刻は、[35182.10204999]
車両が顧客6183に到着する時刻は、[35188.81025392]
車両が顧客6184に到着する時刻は、[35191.81025392]
車両が顧客6185に到着する時刻は、[35194.97253158]
車両が顧客6186に到着する時刻は、[35197.80095871]
車両が顧客6187に到着する時刻は、[35207.02050316]
車両が顧客6188に到着する時刻は、[35208.43471673]
車両が顧客6189に到着する時刻は、[35217.37898864]
車両が顧客6190に到着する時刻は、[35227.42886426]
車両が顧客6191に到着する時刻は、[35230.42886426]
車両が顧客6192に到着する時刻は、[35241.24551808]
車両が顧客6193に到着する時刻は、[35250.67949922]
車両が顧客6194に到着する時刻は、[35257.38770315]
車両が顧客6195に到着する時刻は、[35260.54998081]
車両が顧客6196に到着する時刻は、[35265.54998081]
車両が顧客6197に到着する時刻は、[35271.63274334]
車両が顧客6198に到着する時刻は、[3

In [ ]:
for var in whole_x.getVars():
    print(var.varName, var.X)

In [ ]:
for constr in whole_x.getConstrs():
    print(constr.Pi)

## ②前半と後半をつなげるveb.
1. 適当なところで前後に分ける
1. 前半後半それぞれのPrimalを解く
1. 前半と後半それぞれのPrimalの最適解を、全体のDualに入れて解く
1. 全体のPrimalの最適解を得る

### 1. 適当なところで前後に分ける
- ひとまず半分くらいで分けることにする

In [13]:
threshold = len(tour)//2
#print(threshold)

former = tour[:threshold]
latter = tour[threshold:]
#print(f"巡回路全体は、{tour}")
print(f"前半は、{former}")
print(f"後半は、{latter}")

前半は、[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 

### 2. 前半後半それぞれのPrimalを解く

In [14]:
import numpy as np
# 係数行列、ベクトルを整える
A1x, A1p, A2x, A2p, A3x, A3p, b1, b2, b3, c1, c2 = [], [], [], [], [], [], [], [], [], [], []
## A1, A2, A3を定める
## b1, b2, b3を定める
for Ax_i, Ap_i, b_i in zip(Ax, Ap, b):
    if np.linalg.norm(Ax_i[threshold:], ord=2)==0.:
        A1x.append(Ax_i[:threshold])
        A1p.append(Ap_i[:threshold])
        b1.append(b_i)
    elif np.linalg.norm(Ax_i[:threshold], ord=2)==0.:
        A2x.append(Ax_i[threshold:])
        A2p.append(Ap_i[threshold:])
        b2.append(b_i)
    else:
        A3x.append(Ax_i)
        A3p.append(Ap_i)
        b3.append(b_i)
## c1, c2, c3を定める
c1 = c[:threshold]
c2 = c[threshold:]
## リストからnumpy arrayに変換
A1x = np.array(A1x)
A1p = np.array(A1p)
A2x = np.array(A2x)
A2p = np.array(A2p)
A3x = np.array(A3x)
A3p = np.array(A3p)
b1 = np.array(b1)
b2 = np.array(b2)
b3 = np.array(b3)
c1 = np.array(c1)
c2 = np.array(c2)
for i in range(1, 4):
    print(f"A{i}x : ", end="")
    print(eval("A"+str(i)+"x"))
    print(f"A{i}p : ", end="")
    print(eval("A"+str(i)+"p"))
    print(f"b{i} : ", end="")
    print(eval("b"+str(i)))
    if i <= 2:
        print(eval("c"+str(i)))

A1x : [[ 1 -1  0 ...  0  0  0]
 [ 0  1 -1 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0  1]]
A1p : [[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0 -1]]
b1 : [-6.08276253e+00 -3.60555128e+00 -5.83095189e+00 ...  1.10180000e+04
 -1.10190000e+04  1.10230000e+04]
[1 1 1 ... 1 1 1]
A2x : [[ 1 -1  0 ...  0  0  0]
 [ 0  1 -1 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0  1]]
A2p : [[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0 -1]]
b2 : [-1.00000000e+01 -6.70820393e+00 -8.06225775e+00 ...  2.20510000e+04
  0.00000000e+00  1.00000000e+03]
[1 1 1 ... 1 1 1]
A3x : [[0 0 0 ... 0 0 0]]
A3p : [[0 0 0 ... 0 0 0]]
b3 : [-8.06225775]


In [15]:
# Gurobiによって最適解を求める
former_x = solve_primal(A1x, A1p, b1, c1, instance_name+"former_x", len(former))
latter_x = solve_primal(A2x, A2p, b2, c2, instance_name+"latter_x", len(latter))

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 9482 rows, 6322 columns and 18964 nonzeros
Model fingerprint: 0x79f6c9b9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 6294 rows and 3136 columns
Presolve time: 0.05s
Presolved: 3188 rows, 3186 columns, 6376 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0943210e+07   4.794063e+00   0.000000e+00      0s
    2980    1.0954124e+07   0.000000e+00   0.000000e+00      0s

Solved in 2980 iterations and 0.17 seconds
Optimal objective  1.095412389e+07
車両が顧客0に到着する時刻は、[0.]
車両が顧客1に到着する時刻は、[6.08276253]
車両が顧客2に到着する時刻は、[9.68831381]
車両が顧客3に到着する時刻は、[15.5192657]
車両が顧客4に到着する時刻は、[22.5192657]
車両が顧客5に到着する時刻は、[24.5192657]
車両が顧客6に到着する時刻は、[35.8329742]
車両が顧客7に到着する時刻は、[47.23472845]
車両が顧客8に到着する時刻は、[55.7787322]
車両が顧客9に到着する時刻は、[59.38428347]
車両が顧客10に到着する時刻は、[67.44654122]

車両が顧客237に到着する時刻は、[1390.26410443]
車両が顧客238に到着する時刻は、[1400.55973457]
車両が顧客239に到着する時刻は、[1409.99371571]
車両が顧客240に到着する時刻は、[1413.15599337]
車両が顧客241に到着する時刻は、[1417.15599337]
車両が顧客242に到着する時刻は、[1421.27909899]
車両が顧客243に到着する時刻は、[1426.27909899]
車両が顧客244に到着する時刻は、[1432.60365431]
車両が顧客245に到着する時刻は、[1438.92820963]
車両が顧客246に到着する時刻は、[1442.92820963]
車両が顧客247に到着する時刻は、[1447.92820963]
車両が顧客248に到着する時刻は、[1454.63641357]
車両が顧客249に到着する時刻は、[1466.29831736]
車両が顧客250に到着する時刻は、[1476.49635638]
車両が顧客251に到着する時刻は、[1481.88152119]
車両が顧客252に到着する時刻は、[1491.10106565]
車両が顧客253に到着する時刻は、[1497.50418988]
車両が顧客254に到着する時刻は、[1501.62729551]
車両が顧客255に到着する時刻は、[1511.67717113]
車両が顧客256に到着する時刻は、[1513.91323911]
車両が顧客257に到着する時刻は、[1516.74166623]
車両が顧客258に到着する時刻は、[1523.74166623]
車両が顧客259に到着する時刻は、[1529.12683104]
車両が顧客260に到着する時刻は、[1538.56081217]
車両が顧客261に到着する時刻は、[1549.33114179]
車両が顧客262に到着する時刻は、[1551.33114179]
車両が顧客263に到着する時刻は、[1555.57378247]
車両が顧客264に到着する時刻は、[1562.78488503]
車両が顧客265に到着する時刻は、[1568.86764756]
車両が顧客266に到着する時刻は、[1576.9299053]
車両が顧客267に到着

車両が顧客586に到着する時刻は、[3380.04267668]
車両が顧客587に到着する時刻は、[3387.85292636]
車両が顧客588に到着する時刻は、[3394.92399417]
車両が顧客589に到着する時刻は、[3395.92399417]
車両が顧客590に到着する時刻は、[3402.0067567]
車両が顧客591に到着する時刻は、[3411.0067567]
車両が顧客592に到着する時刻は、[3418.07782451]
車両が顧客593に到着する時刻は、[3427.51180564]
車両が顧客594に到着する時刻は、[3435.12757875]
車両が顧客595に到着する時刻は、[3438.12757875]
車両が顧客596に到着する時刻は、[3444.53070299]
車両が顧客597に到着する時刻は、[3450.18755724]
車両が顧客598に到着する時刻は、[3459.40710169]
車両が顧客599に到着する時刻は、[3462.23552882]
車両が顧客600に到着する時刻は、[3465.06395594]
車両が顧客601に到着する時刻は、[3471.14671847]
車両が顧客602に到着する時刻は、[3477.229481]
車両が顧客603に到着する時刻は、[3484.30054882]
車両が顧客604に到着する時刻は、[3492.84455256]
車両が顧客605に到着する時刻は、[3498.92731509]
車両が顧客606に到着する時刻は、[3504.3124799]
車両が顧客607に到着する時刻は、[3510.63703522]
車両が顧客608に到着する時刻は、[3521.07734173]
車両が顧客609に到着する時刻は、[3523.31340971]
車両が顧客610に到着する時刻は、[3524.72762327]
車両が顧客611に到着する時刻は、[3528.33317454]
車両が顧客612に到着する時刻は、[3538.62880468]
車両が顧客613に到着する時刻は、[3547.68418982]
車両が顧客614に到着する時刻は、[3550.51261695]
車両が顧客615に到着する時刻は、[3557.51261695]
車両が顧客616に到着する時刻

車両が顧客843に到着する時刻は、[4777.47861463]
車両が顧客844に到着する時刻は、[4783.56137716]
車両が顧客845に到着する時刻は、[4791.37162683]
車両が顧客846に到着する時刻は、[4798.65173672]
車両が顧客847に到着する時刻は、[4806.89794797]
車両が顧客848に到着する時刻は、[4813.96901579]
車両が顧客849に到着する時刻は、[4818.21165647]
車両が顧客850に到着する時刻は、[4821.81720775]
車両が顧客851に到着する時刻は、[4829.8794655]
車両が顧客852に到着する時刻は、[4831.29367906]
車両が顧客853に到着する時刻は、[4838.29367906]
車両が顧客854に到着する時刻は、[4841.45595672]
車両が顧客855に到着する時刻は、[4843.6920247]
車両が顧客856に到着する時刻は、[4849.6920247]
車両が顧客857に到着する時刻は、[4856.76309251]
車両が顧客858に到着する時刻は、[4863.83416032]
車両が顧客859に到着する時刻は、[4874.46430613]
車両が顧客860に到着する時刻は、[4881.17251007]
車両が顧客861に到着する時刻は、[4888.24357788]
車両が顧客862に到着する時刻は、[4892.48621856]
車両が顧客863に到着する時刻は、[4892.48621856]
車両が顧客864に到着する時刻は、[4895.64849622]
車両が顧客865に到着する時刻は、[4903.64849622]
車両が顧客866に到着する時刻は、[4912.86804068]
車両が顧客867に到着する時刻は、[4918.69899258]
車両が顧客868に到着する時刻は、[4927.24299632]
車両が顧客869に到着する時刻は、[4935.48920757]
車両が顧客870に到着する時刻は、[4941.32015947]
車両が顧客871に到着する時刻は、[4945.32015947]
車両が顧客872に到着する時刻は、[4949.56280016]
車両が顧客873に到着する

車両が顧客1249に到着する時刻は、[7054.43105225]
車両が顧客1250に到着する時刻は、[7062.04682536]
車両が顧客1251に到着する時刻は、[7065.04682536]
車両が顧客1252に到着する時刻は、[7067.28289334]
車両が顧客1253に到着する時刻は、[7072.38191285]
車両が顧客1254に到着する時刻は、[7079.45298066]
車両が顧客1255に到着する時刻は、[7082.61525832]
車両が顧客1256に到着する時刻は、[7091.21758359]
車両が顧客1257に到着する時刻は、[7094.04601071]
車両が顧客1258に到着する時刻は、[7099.04601071]
車両が顧客1259に到着する時刻は、[7104.43117552]
車両が顧客1260に到着する時刻は、[7105.84538908]
車両が顧客1261に到着する時刻は、[7111.23055389]
車両が顧客1262に到着する時刻は、[7114.39283155]
車両が顧客1263に到着する時刻は、[7119.39283155]
車両が顧客1264に到着する時刻は、[7124.39283155]
車両が顧客1265に到着する時刻は、[7128.51593718]
車両が顧客1266に到着する時刻は、[7130.51593718]
車両が顧客1267に到着する時刻は、[7135.90110198]
車両が顧客1268に到着する時刻は、[7139.06337964]
車両が顧客1269に到着する時刻は、[7146.87362932]
車両が顧客1270に到着する時刻は、[7148.87362932]
車両が顧客1271に到着する時刻は、[7152.47918059]
車両が顧客1272に到着する時刻は、[7159.75929048]
車両が顧客1273に到着する時刻は、[7171.16104473]
車両が顧客1274に到着する時刻は、[7181.79119055]
車両が顧客1275に到着する時刻は、[7184.02725852]
車両が顧客1276に到着する時刻は、[7190.11002106]
車両が顧客1277に到着する時刻は、[7197.11002106]
車両が顧客1278に到着する

...

車両が顧客2807に到着する時刻は、[16013.97812809]
車両が顧客2808に到着する時刻は、[16015.39234166]
車両が顧客2809に到着する時刻は、[16023.20259133]
車両が顧客2810に到着する時刻は、[16024.20259133]
車両が顧客2811に到着する時刻は、[16027.03101846]
車両が顧客2812に到着する時刻は、[16031.27365914]
車両が顧客2813に到着する時刻は、[16038.88943225]
車両が顧客2814に到着する時刻は、[16043.01253788]
車両が顧客2815に到着する時刻は、[16049.72074181]
車両が顧客2816に到着する時刻は、[16057.53099148]
車両が顧客2817に到着する時刻は、[16063.61375401]
車両が顧客2818に到着する時刻は、[16066.61375401]
車両が顧客2819に到着する時刻は、[16071.99891882]
車両が顧客2820に到着する時刻は、[16078.08168135]
車両が顧客2821に到着する時刻は、[16079.49589491]
車両が顧客2822に到着する時刻は、[16087.11166802]
車両が顧客2823に到着する時刻は、[16095.65567177]
車両が顧客2824に到着する時刻は、[16102.72673958]
車両が顧客2825に到着する時刻は、[16108.55769147]
車両が顧客2826に到着する時刻は、[16110.79375945]
車両が顧客2827に到着する時刻は、[16120.01330391]
車両が顧客2828に到着する時刻は、[16122.01330391]
車両が顧客2829に到着する時刻は、[16125.61885518]
車両が顧客2830に到着する時刻は、[16128.78113284]
車両が顧客2831に到着する時刻は、[16133.88015236]
車両が顧客2832に到着する時刻は、[16137.04243002]
車両が顧客2833に到着する時刻は、[16142.87338191]
車両が顧客2834に到着する時刻は、[16148.25854672]
車両が顧客2835に到着する時刻は、[1

車両が顧客3042に到着する時刻は、[17334.72768922]
車両が顧客3043に到着する時刻は、[17335.72768922]
車両が顧客3044に到着する時刻は、[17338.72768922]
車両が顧客3045に到着する時刻は、[17342.9703299]
車両が顧客3046に到着する時刻は、[17350.25043979]
車両が顧客3047に到着する時刻は、[17355.25043979]
車両が顧客3048に到着する時刻は、[17361.95864372]
車両が顧客3049に到着する時刻は、[17363.37285729]
車両が顧客3050に到着する時刻は、[17368.37285729]
車両が顧客3051に到着する時刻は、[17370.60892526]
車両が顧客3052に到着する時刻は、[17372.84499324]
車両が顧客3053に到着する時刻は、[17380.12510313]
車両が顧客3054に到着する時刻は、[17385.12510313]
車両が顧客3055に到着する時刻は、[17393.7274284]
車両が顧客3056に到着する時刻は、[17401.97363965]
車両が顧客3057に到着する時刻は、[17404.80206677]
車両が顧客3058に到着する時刻は、[17412.08217666]
車両が顧客3059に到着する時刻は、[17413.08217666]
車両が顧客3060に到着する時刻は、[17419.4853009]
車両が顧客3061に到着する時刻は、[17426.55636871]
車両が顧客3062に到着する時刻は、[17431.94153352]
車両が顧客3063に到着する時刻は、[17435.5470848]
車両が顧客3064に到着する時刻は、[17445.5470848]
車両が顧客3065に到着する時刻は、[17456.17723061]
車両が顧客3066に到着する時刻は、[17461.56239542]
車両が顧客3067に到着する時刻は、[17466.56239542]
車両が顧客3068に到着する時刻は、[17473.63346323]
車両が顧客3069に到着する時刻は、[17480.84456578]
車両が顧客3070に到着する時刻は、[17486.

車両が顧客1003に到着する時刻は、[13252.33904357]
車両が顧客1004に到着する時刻は、[13257.43806309]
車両が顧客1005に到着する時刻は、[13261.04361436]
車両が顧客1006に到着する時刻は、[13270.9431093]
車両が顧客1007に到着する時刻は、[13280.9431093]
車両が顧客1008に到着する時刻は、[13291.14114833]
車両が顧客1009に到着する時刻は、[13293.96957545]
車両が顧客1010に到着する時刻は、[13298.09268108]
車両が顧客1011に到着する時刻は、[13307.94153888]
車両が顧客1012に到着する時刻は、[13312.0646445]
車両が顧客1013に到着する時刻は、[13316.30728519]
車両が顧客1014に到着する時刻は、[13323.378353]
車両が顧客1015に到着する時刻は、[13326.54063066]
車両が顧客1016に到着する時刻は、[13327.95484422]
車両が顧客1017に到着する時刻は、[13337.95484422]
車両が顧客1018に到着する時刻は、[13347.95484422]
車両が顧客1019に到着する時刻は、[13350.1909122]
車両が顧客1020に到着する時刻は、[13353.79646348]
車両が顧客1021に到着する時刻は、[13357.9195691]
車両が顧客1022に到着する時刻は、[13364.32269334]
車両が顧客1023に到着する時刻は、[13370.64724866]
車両が顧客1024に到着する時刻は、[13372.06146222]
車両が顧客1025に到着する時刻は、[13380.5467436]
車両が顧客1026に到着する時刻は、[13382.78281157]
車両が顧客1027に到着する時刻は、[13388.78281157]
車両が顧客1028に到着する時刻は、[13397.38513684]
車両が顧客1029に到着する時刻は、[13404.09334077]
車両が顧客1030に到着する時刻は、[13413.03761268]
車両が顧客1031に到着する時刻は、[13415.037

車両が顧客1974に到着する時刻は、[18799.27709048]
車両が顧客1975に到着する時刻は、[18808.49663494]
車両が顧客1976に到着する時刻は、[18808.49663494]
車両が顧客1977に到着する時刻は、[18817.04063868]
車両が顧客1978に到着する時刻は、[18823.12340121]
車両が顧客1979に到着する時刻は、[18829.52652545]
車両が顧客1980に到着する時刻は、[18833.99866141]
車両が顧客1981に到着する時刻は、[18839.8296133]
車両が顧客1982に到着する時刻は、[18849.72910824]
車両が顧客1983に到着する時刻は、[18853.97174893]
車両が顧客1984に到着する時刻は、[18858.44388488]
車両が顧客1985に到着する時刻は、[18861.44388488]
車両が顧客1986に到着する時刻は、[18869.98788863]
車両が顧客1987に到着する時刻は、[18878.59021389]
車両が顧客1988に到着する時刻は、[18889.0305204]
車両が顧客1989に到着する時刻は、[18897.09277815]
車両が顧客1990に到着する時刻は、[18904.70855126]
車両が顧客1991に到着する時刻は、[18913.76393639]
車両が顧客1992に到着する時刻は、[18920.83500421]
車両が顧客1993に到着する時刻は、[18923.07107218]
車両が顧客1994に到着する時刻は、[18929.15383471]
車両が顧客1995に到着する時刻は、[18937.69783846]
車両が顧客1996に到着する時刻は、[18946.30016373]
車両が顧客1997に到着する時刻は、[18955.24443564]
車両が顧客1998に到着する時刻は、[18959.24443564]
車両が顧客1999に到着する時刻は、[18964.34345515]
車両が顧客2000に到着する時刻は、[18975.16010898]
車両が顧客2001に到着する時刻は、[18986.82201277]
車両が顧客2002に到着する時刻は、[189

車両が顧客2543に到着する時刻は、[22156.85784413]
車両が顧客2544に到着する時刻は、[22160.02012179]
車両が顧客2545に到着する時刻は、[22162.84854891]
車両が顧客2546に到着する時刻は、[22165.08461689]
車両が顧客2547に到着する時刻は、[22169.20772251]
車両が顧客2548に到着する時刻は、[22172.81327379]
車両が顧客2549に到着する時刻は、[22175.81327379]
車両が顧客2550に到着する時刻は、[22182.52147772]
車両が顧客2551に到着する時刻は、[22189.22968165]
車両が顧客2552に到着する時刻は、[22195.55423698]
車両が顧客2553に到着する時刻は、[22203.61649472]
車両が顧客2554に到着する時刻は、[22213.46535253]
車両が顧客2555に到着する時刻は、[22219.78990785]
車両が顧客2556に到着する時刻は、[22221.20412141]
車両が顧客2557に到着する時刻は、[22228.81989451]
車両が顧客2558に到着する時刻は、[22236.88215226]
車両が顧客2559に到着する時刻は、[22241.35428822]
車両が顧客2560に到着する時刻は、[22249.16453789]
車両が顧客2561に到着する時刻は、[22257.22679564]
車両が顧客2562に到着する時刻は、[22261.34990127]
車両が顧客2563に到着する時刻は、[22267.00675552]
車両が顧客2564に到着する時刻は、[22269.00675552]
車両が顧客2565に到着する時刻は、[22271.00675552]
車両が顧客2566に到着する時刻は、[22282.40850977]
車両が顧客2567に到着する時刻は、[22290.65472102]
車両が顧客2568に到着する時刻は、[22297.65472102]
車両が顧客2569に到着する時刻は、[22303.73748355]
車両が顧客2570に到着する時刻は、[22309.12264836]
車両が顧客2571に到着する時刻は、[2

In [ ]:
for v in former_x.getVars():
    print('%s %g %g' % (v.varName, v.x, v.VBasis))

In [ ]:
for v in latter_x.getVars():
    print('%s %g %g' % (v.varName, v.x, v.VBasis))

In [ ]:
for constr in former_x.getConstrs()+latter_x.getConstrs():
    print(constr.Pi, constr.CBasis)

### 3. 前半後半それぞれのPrimalの最適解を、全体のDualに入れて解く

In [16]:
# 初期解の保存
basis_like = np.array([constr.Pi for constr in former_x.getConstrs()+latter_x.getConstrs()])
y3 = np.zeros((Ax.shape[0]-basis_like.shape[0],))
basis_like = np.append(basis_like, y3)

In [ ]:
print(f"Ax.T.shape={Ax.T.shape}\t\t\tAp.T.shape={Ap.T.shape}")
for Ax_i, Ap_i in zip(Ax.T, Ap.T):
    print(Ax_i, "\t", Ap_i)

In [17]:
# Gurobiによって最適解を求める
whole_y = solve_dual(Ax, Ap, b, c, instance_name+"whole_y", basis_like.shape[0], basis_like)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 31609 rows, 18965 columns and 56895 nonzeros
Model fingerprint: 0xd26099ab
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 31602 rows and 18956 columns
Presolve time: 0.12s
Presolved: 7 rows, 9 columns, 18 nonzeros

Ordering time: 0.00s

Barrier performed 0 iterations in 0.13 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 2 iterations and 0.15 seconds
Optimal objective  4.388760208e+07
主問題における制約0の潜在価格は、[6317.]
主問題における制約1の潜在価格は、[6318.]
主問題における制約2の潜在価格は、[6319.]
主問題における制約3の潜在価格は、[6318.]
主問題における制約4の潜在価格は、[6317.]
主問題における制約5の潜在価格は、[6316.]
主問題における制約6の潜在価格は、[6315.]
主問題における制約7の潜在価格は、[6314.]
主問題における制約8の潜在価格は、[6313.]
主問題における制約9の潜在価格は、[6312.]
主問題における制約10の潜在価

主問題における制約624の潜在価格は、[5697.]
主問題における制約625の潜在価格は、[5696.]
主問題における制約626の潜在価格は、[5695.]
主問題における制約627の潜在価格は、[5694.]
主問題における制約628の潜在価格は、[5693.]
主問題における制約629の潜在価格は、[5692.]
主問題における制約630の潜在価格は、[5691.]
主問題における制約631の潜在価格は、[5690.]
主問題における制約632の潜在価格は、[5689.]
主問題における制約633の潜在価格は、[5688.]
主問題における制約634の潜在価格は、[5687.]
主問題における制約635の潜在価格は、[5686.]
主問題における制約636の潜在価格は、[5685.]
主問題における制約637の潜在価格は、[5684.]
主問題における制約638の潜在価格は、[5683.]
主問題における制約639の潜在価格は、[5682.]
主問題における制約640の潜在価格は、[5681.]
主問題における制約641の潜在価格は、[5680.]
主問題における制約642の潜在価格は、[5679.]
主問題における制約643の潜在価格は、[5678.]
主問題における制約644の潜在価格は、[5677.]
主問題における制約645の潜在価格は、[5676.]
主問題における制約646の潜在価格は、[5675.]
主問題における制約647の潜在価格は、[5674.]
主問題における制約648の潜在価格は、[5673.]
主問題における制約649の潜在価格は、[5672.]
主問題における制約650の潜在価格は、[5671.]
主問題における制約651の潜在価格は、[5670.]
主問題における制約652の潜在価格は、[5669.]
主問題における制約653の潜在価格は、[5668.]
主問題における制約654の潜在価格は、[5667.]
主問題における制約655の潜在価格は、[5666.]
主問題における制約656の潜在価格は、[5665.]
主問題における制約657の潜在価格は、[5664.]
主問題における制約658の潜在価格は、[5663.]
主問題における制約659の潜在価格は、[5662.]
主問題における制約660の潜在価格は、[5661.]
主

主問題における制約1266の潜在価格は、[5055.]
主問題における制約1267の潜在価格は、[5054.]
主問題における制約1268の潜在価格は、[5053.]
主問題における制約1269の潜在価格は、[5052.]
主問題における制約1270の潜在価格は、[5051.]
主問題における制約1271の潜在価格は、[5050.]
主問題における制約1272の潜在価格は、[5049.]
主問題における制約1273の潜在価格は、[5048.]
主問題における制約1274の潜在価格は、[5047.]
主問題における制約1275の潜在価格は、[5046.]
主問題における制約1276の潜在価格は、[5045.]
主問題における制約1277の潜在価格は、[5044.]
主問題における制約1278の潜在価格は、[5043.]
主問題における制約1279の潜在価格は、[5042.]
主問題における制約1280の潜在価格は、[5041.]
主問題における制約1281の潜在価格は、[5040.]
主問題における制約1282の潜在価格は、[5039.]
主問題における制約1283の潜在価格は、[5038.]
主問題における制約1284の潜在価格は、[5037.]
主問題における制約1285の潜在価格は、[5036.]
主問題における制約1286の潜在価格は、[5035.]
主問題における制約1287の潜在価格は、[5034.]
主問題における制約1288の潜在価格は、[5033.]
主問題における制約1289の潜在価格は、[5032.]
主問題における制約1290の潜在価格は、[5031.]
主問題における制約1291の潜在価格は、[5030.]
主問題における制約1292の潜在価格は、[5029.]
主問題における制約1293の潜在価格は、[5028.]
主問題における制約1294の潜在価格は、[5027.]
主問題における制約1295の潜在価格は、[5026.]
主問題における制約1296の潜在価格は、[5025.]
主問題における制約1297の潜在価格は、[5024.]
主問題における制約1298の潜在価格は、[5023.]
主問題における制約1299の潜在価格は、[5022.]
主問題における制約1300の潜在価格は、[5021.]
主問題における制約1301の潜在価格は、

主問題における制約1575の潜在価格は、[4746.]
主問題における制約1576の潜在価格は、[4745.]
主問題における制約1577の潜在価格は、[4744.]
主問題における制約1578の潜在価格は、[4743.]
主問題における制約1579の潜在価格は、[4742.]
主問題における制約1580の潜在価格は、[4741.]
主問題における制約1581の潜在価格は、[4740.]
主問題における制約1582の潜在価格は、[4739.]
主問題における制約1583の潜在価格は、[4738.]
主問題における制約1584の潜在価格は、[4737.]
主問題における制約1585の潜在価格は、[4736.]
主問題における制約1586の潜在価格は、[4735.]
主問題における制約1587の潜在価格は、[4734.]
主問題における制約1588の潜在価格は、[4733.]
主問題における制約1589の潜在価格は、[4732.]
主問題における制約1590の潜在価格は、[4731.]
主問題における制約1591の潜在価格は、[4730.]
主問題における制約1592の潜在価格は、[4729.]
主問題における制約1593の潜在価格は、[4728.]
主問題における制約1594の潜在価格は、[4727.]
主問題における制約1595の潜在価格は、[4726.]
主問題における制約1596の潜在価格は、[4725.]
主問題における制約1597の潜在価格は、[4724.]
主問題における制約1598の潜在価格は、[4723.]
主問題における制約1599の潜在価格は、[4722.]
主問題における制約1600の潜在価格は、[4721.]
主問題における制約1601の潜在価格は、[4720.]
主問題における制約1602の潜在価格は、[4719.]
主問題における制約1603の潜在価格は、[4718.]
主問題における制約1604の潜在価格は、[4717.]
主問題における制約1605の潜在価格は、[4716.]
主問題における制約1606の潜在価格は、[4715.]
主問題における制約1607の潜在価格は、[4714.]
主問題における制約1608の潜在価格は、[4713.]
主問題における制約1609の潜在価格は、[4712.]
主問題における制約1610の潜在価格は、

主問題における制約1906の潜在価格は、[4415.]
主問題における制約1907の潜在価格は、[4414.]
主問題における制約1908の潜在価格は、[4413.]
主問題における制約1909の潜在価格は、[4412.]
主問題における制約1910の潜在価格は、[4411.]
主問題における制約1911の潜在価格は、[4410.]
主問題における制約1912の潜在価格は、[4409.]
主問題における制約1913の潜在価格は、[4408.]
主問題における制約1914の潜在価格は、[4407.]
主問題における制約1915の潜在価格は、[4406.]
主問題における制約1916の潜在価格は、[4405.]
主問題における制約1917の潜在価格は、[4404.]
主問題における制約1918の潜在価格は、[4403.]
主問題における制約1919の潜在価格は、[4402.]
主問題における制約1920の潜在価格は、[4401.]
主問題における制約1921の潜在価格は、[4400.]
主問題における制約1922の潜在価格は、[4399.]
主問題における制約1923の潜在価格は、[4398.]
主問題における制約1924の潜在価格は、[4397.]
主問題における制約1925の潜在価格は、[4396.]
主問題における制約1926の潜在価格は、[4395.]
主問題における制約1927の潜在価格は、[4394.]
主問題における制約1928の潜在価格は、[4393.]
主問題における制約1929の潜在価格は、[4392.]
主問題における制約1930の潜在価格は、[4391.]
主問題における制約1931の潜在価格は、[4390.]
主問題における制約1932の潜在価格は、[4389.]
主問題における制約1933の潜在価格は、[4388.]
主問題における制約1934の潜在価格は、[4387.]
主問題における制約1935の潜在価格は、[4386.]
主問題における制約1936の潜在価格は、[4385.]
主問題における制約1937の潜在価格は、[4384.]
主問題における制約1938の潜在価格は、[4383.]
主問題における制約1939の潜在価格は、[4382.]
主問題における制約1940の潜在価格は、[4381.]
主問題における制約1941の潜在価格は、

...

主問題における制約15996の潜在価格は、[1.]
主問題における制約15997の潜在価格は、[0.]
主問題における制約15998の潜在価格は、[1.]
主問題における制約15999の潜在価格は、[0.]
主問題における制約16000の潜在価格は、[1.]
主問題における制約16001の潜在価格は、[0.]
主問題における制約16002の潜在価格は、[1.]
主問題における制約16003の潜在価格は、[0.]
主問題における制約16004の潜在価格は、[1.]
主問題における制約16005の潜在価格は、[0.]
主問題における制約16006の潜在価格は、[1.]
主問題における制約16007の潜在価格は、[0.]
主問題における制約16008の潜在価格は、[1.]
主問題における制約16009の潜在価格は、[0.]
主問題における制約16010の潜在価格は、[1.]
主問題における制約16011の潜在価格は、[0.]
主問題における制約16012の潜在価格は、[1.]
主問題における制約16013の潜在価格は、[0.]
主問題における制約16014の潜在価格は、[1.]
主問題における制約16015の潜在価格は、[0.]
主問題における制約16016の潜在価格は、[1.]
主問題における制約16017の潜在価格は、[0.]
主問題における制約16018の潜在価格は、[1.]
主問題における制約16019の潜在価格は、[0.]
主問題における制約16020の潜在価格は、[1.]
主問題における制約16021の潜在価格は、[0.]
主問題における制約16022の潜在価格は、[1.]
主問題における制約16023の潜在価格は、[0.]
主問題における制約16024の潜在価格は、[1.]
主問題における制約16025の潜在価格は、[0.]
主問題における制約16026の潜在価格は、[1.]
主問題における制約16027の潜在価格は、[0.]
主問題における制約16028の潜在価格は、[1.]
主問題における制約16029の潜在価格は、[0.]
主問題における制約16030の潜在価格は、[1.]
主問題における制約16031の潜在価格は、[0.]
主問題における制約16032の潜在価格は、[1.]
主問題における制約16033の潜在価格は、[0.]
主問題における制約160

主問題における制約16680の潜在価格は、[1.]
主問題における制約16681の潜在価格は、[0.]
主問題における制約16682の潜在価格は、[1.]
主問題における制約16683の潜在価格は、[0.]
主問題における制約16684の潜在価格は、[1.]
主問題における制約16685の潜在価格は、[0.]
主問題における制約16686の潜在価格は、[1.]
主問題における制約16687の潜在価格は、[0.]
主問題における制約16688の潜在価格は、[1.]
主問題における制約16689の潜在価格は、[0.]
主問題における制約16690の潜在価格は、[1.]
主問題における制約16691の潜在価格は、[0.]
主問題における制約16692の潜在価格は、[1.]
主問題における制約16693の潜在価格は、[0.]
主問題における制約16694の潜在価格は、[1.]
主問題における制約16695の潜在価格は、[0.]
主問題における制約16696の潜在価格は、[1.]
主問題における制約16697の潜在価格は、[0.]
主問題における制約16698の潜在価格は、[1.]
主問題における制約16699の潜在価格は、[0.]
主問題における制約16700の潜在価格は、[1.]
主問題における制約16701の潜在価格は、[0.]
主問題における制約16702の潜在価格は、[1.]
主問題における制約16703の潜在価格は、[0.]
主問題における制約16704の潜在価格は、[1.]
主問題における制約16705の潜在価格は、[0.]
主問題における制約16706の潜在価格は、[1.]
主問題における制約16707の潜在価格は、[0.]
主問題における制約16708の潜在価格は、[1.]
主問題における制約16709の潜在価格は、[0.]
主問題における制約16710の潜在価格は、[1.]
主問題における制約16711の潜在価格は、[0.]
主問題における制約16712の潜在価格は、[1.]
主問題における制約16713の潜在価格は、[0.]
主問題における制約16714の潜在価格は、[1.]
主問題における制約16715の潜在価格は、[0.]
主問題における制約16716の潜在価格は、[1.]
主問題における制約16717の潜在価格は、[0.]
主問題における制約167

主問題における制約17268の潜在価格は、[1.]
主問題における制約17269の潜在価格は、[0.]
主問題における制約17270の潜在価格は、[1.]
主問題における制約17271の潜在価格は、[0.]
主問題における制約17272の潜在価格は、[1.]
主問題における制約17273の潜在価格は、[0.]
主問題における制約17274の潜在価格は、[1.]
主問題における制約17275の潜在価格は、[0.]
主問題における制約17276の潜在価格は、[1.]
主問題における制約17277の潜在価格は、[0.]
主問題における制約17278の潜在価格は、[1.]
主問題における制約17279の潜在価格は、[0.]
主問題における制約17280の潜在価格は、[1.]
主問題における制約17281の潜在価格は、[0.]
主問題における制約17282の潜在価格は、[1.]
主問題における制約17283の潜在価格は、[0.]
主問題における制約17284の潜在価格は、[1.]
主問題における制約17285の潜在価格は、[0.]
主問題における制約17286の潜在価格は、[1.]
主問題における制約17287の潜在価格は、[0.]
主問題における制約17288の潜在価格は、[1.]
主問題における制約17289の潜在価格は、[0.]
主問題における制約17290の潜在価格は、[1.]
主問題における制約17291の潜在価格は、[0.]
主問題における制約17292の潜在価格は、[1.]
主問題における制約17293の潜在価格は、[0.]
主問題における制約17294の潜在価格は、[1.]
主問題における制約17295の潜在価格は、[0.]
主問題における制約17296の潜在価格は、[1.]
主問題における制約17297の潜在価格は、[0.]
主問題における制約17298の潜在価格は、[1.]
主問題における制約17299の潜在価格は、[0.]
主問題における制約17300の潜在価格は、[1.]
主問題における制約17301の潜在価格は、[0.]
主問題における制約17302の潜在価格は、[1.]
主問題における制約17303の潜在価格は、[0.]
主問題における制約17304の潜在価格は、[1.]
主問題における制約17305の潜在価格は、[0.]
主問題における制約173

主問題における制約18321の潜在価格は、[0.]
主問題における制約18322の潜在価格は、[1.]
主問題における制約18323の潜在価格は、[0.]
主問題における制約18324の潜在価格は、[1.]
主問題における制約18325の潜在価格は、[0.]
主問題における制約18326の潜在価格は、[1.]
主問題における制約18327の潜在価格は、[0.]
主問題における制約18328の潜在価格は、[1.]
主問題における制約18329の潜在価格は、[0.]
主問題における制約18330の潜在価格は、[1.]
主問題における制約18331の潜在価格は、[0.]
主問題における制約18332の潜在価格は、[1.]
主問題における制約18333の潜在価格は、[0.]
主問題における制約18334の潜在価格は、[1.]
主問題における制約18335の潜在価格は、[0.]
主問題における制約18336の潜在価格は、[1.]
主問題における制約18337の潜在価格は、[0.]
主問題における制約18338の潜在価格は、[1.]
主問題における制約18339の潜在価格は、[0.]
主問題における制約18340の潜在価格は、[1.]
主問題における制約18341の潜在価格は、[0.]
主問題における制約18342の潜在価格は、[1.]
主問題における制約18343の潜在価格は、[0.]
主問題における制約18344の潜在価格は、[1.]
主問題における制約18345の潜在価格は、[0.]
主問題における制約18346の潜在価格は、[1.]
主問題における制約18347の潜在価格は、[0.]
主問題における制約18348の潜在価格は、[1.]
主問題における制約18349の潜在価格は、[0.]
主問題における制約18350の潜在価格は、[1.]
主問題における制約18351の潜在価格は、[0.]
主問題における制約18352の潜在価格は、[1.]
主問題における制約18353の潜在価格は、[0.]
主問題における制約18354の潜在価格は、[1.]
主問題における制約18355の潜在価格は、[0.]
主問題における制約18356の潜在価格は、[1.]
主問題における制約18357の潜在価格は、[0.]
主問題における制約18358の潜在価格は、[1.]
主問題における制約183

主問題における制約18636の潜在価格は、[1.]
主問題における制約18637の潜在価格は、[0.]
主問題における制約18638の潜在価格は、[1.]
主問題における制約18639の潜在価格は、[0.]
主問題における制約18640の潜在価格は、[1.]
主問題における制約18641の潜在価格は、[0.]
主問題における制約18642の潜在価格は、[1.]
主問題における制約18643の潜在価格は、[0.]
主問題における制約18644の潜在価格は、[1.]
主問題における制約18645の潜在価格は、[0.]
主問題における制約18646の潜在価格は、[1.]
主問題における制約18647の潜在価格は、[0.]
主問題における制約18648の潜在価格は、[1.]
主問題における制約18649の潜在価格は、[0.]
主問題における制約18650の潜在価格は、[1.]
主問題における制約18651の潜在価格は、[0.]
主問題における制約18652の潜在価格は、[1.]
主問題における制約18653の潜在価格は、[0.]
主問題における制約18654の潜在価格は、[1.]
主問題における制約18655の潜在価格は、[0.]
主問題における制約18656の潜在価格は、[1.]
主問題における制約18657の潜在価格は、[0.]
主問題における制約18658の潜在価格は、[1.]
主問題における制約18659の潜在価格は、[0.]
主問題における制約18660の潜在価格は、[1.]
主問題における制約18661の潜在価格は、[0.]
主問題における制約18662の潜在価格は、[1.]
主問題における制約18663の潜在価格は、[0.]
主問題における制約18664の潜在価格は、[1.]
主問題における制約18665の潜在価格は、[0.]
主問題における制約18666の潜在価格は、[1.]
主問題における制約18667の潜在価格は、[0.]
主問題における制約18668の潜在価格は、[1.]
主問題における制約18669の潜在価格は、[0.]
主問題における制約18670の潜在価格は、[1.]
主問題における制約18671の潜在価格は、[0.]
主問題における制約18672の潜在価格は、[1.]
主問題における制約18673の潜在価格は、[0.]
主問題における制約186

In [ ]:
# 最適解
for var in whole_y.getVars():
    print(var.varName, var.X)

### 4. 全体のPrimalの最適解を得る

In [ ]:
for i, constr in enumerate(whole_y.getConstrs()):
    print(constr.Pi)

In [ ]:
for var in whole_x.getVars():
    print(var.varName, var.X)